In [70]:
def cheker (supply, demand):
    return sum(supply) == sum(demand)

def fict (supply, demand):
    diff = abs(sum(supply) - sum(demand))
    if sum(supply) < sum(demand):
        supply.append(diff)
    else:
        demand.append(diff)

In [ ]:
def north_west_corner (supply, demand):
    supply_copy = supply.copy()
    demand_copy = demand.copy()
    if (cheker (supply, demand) == False):
        fict (supply, demand)
    
    num_suppliers = len(supply_copy)
    num_consumers = len(demand_copy)
    
    result = [[0] * num_consumers for _ in range(num_suppliers)]
    
    i, j = 0, 0
    
    while i < num_suppliers and j < num_consumers:
        
        min_val = min(supply_copy[i], demand_copy[j])
            
        result[i][j] = min_val
        
        supply_copy[i] -= min_val
        demand_copy[j] -= min_val
        
        if supply_copy[i] == 0:
            i += 1
        else:
            j += 1
    
    return result
    
    

In [84]:
import numpy as np
matrix = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])
supply = [30, 40, 50, 20]
demand = [40, 50, 30]
result = north_west_corner(supply, demand)
ans = matrix * result
print(sum(sum(ans)))
print(ans)

650
[[ 30   0   0]
 [ 40 150   0]
 [  0 160 270]
 [  0   0   0]]
